## Step 0 - the goal of the Markov Chain

we are building a third-order markov model.

to do this, we take the input stream of tokens and capture each trigram.
```
e.g.  
(a,b,c,d,a,b,c,a,b,e,d,a) ->
(a,b,c), (b,c,d), (c,d,a), (d,a,b), (a,b,c), (b,c,a), (c,a,b), (a,b,e), (b,e,d), (e,d,a)
0        1        2        3        4        5        6        7        8        9
```
imagine a cup for each unique pair of tokens starting a trigram

in the cup, put the third letter of the trigram

```
(a,b) -> 0:(a,b,c), 4:(a,b,c), 7:(a,b,e)
```

so cup (a,b) gets 2 (c) and 1 (e)

we start in a random state.

say we start in state (a,b)

from here, there is a 66% chance we choose (c) next and a 33% chance we choose (e) next

if we choose (c), follow these steps:
    output the (a)
    create a new bigram from the second element of the state's key (b) plus the choice: (b,c)
    move to state (b,c)
    probabalistically choose an element from state (b,c)
    repeat
    

## Step 1 - tokenize the text

In [28]:
import nltk
from collections import defaultdict

text = "a b c d a b c a b e d a"

counts = defaultdict(lambda: defaultdict(int))

# note we will use nltk sentence tokenizer for real text
tokens = nltk.word_tokenize(text)
print(tokens)

['a', 'b', 'c', 'd', 'a', 'b', 'c', 'a', 'b', 'e', 'd', 'a']


## Step 2 - create token trigrams

we use zip() to accomplish this.

note, in our system, Ramona, we are building our trigrams out of sentences so that we can generate sentences by modelling the beginning and endings of sentences.  If we wanted to process chunks of text larger than a sentence, which could potentially overwhelm the in-memory lists and zip, we would need to process the tokens in batches, and keep track of the last n-2 tokens in each batch and add them to the beginning of the subsequent batch.


In [29]:
trigrams = sorted(list(zip(tokens, tokens[1:], tokens[2:])))  # all dimensions get sorted

print("all the trigrams")
for t in trigrams:
    print(t)
print()

all the trigrams
('a', 'b', 'c')
('a', 'b', 'c')
('a', 'b', 'e')
('b', 'c', 'a')
('b', 'c', 'd')
('b', 'e', 'd')
('c', 'a', 'b')
('c', 'd', 'a')
('d', 'a', 'b')
('e', 'd', 'a')



## Step 3 - Count the continuation element for each bigram state

make a state for each bigram in our corpus.  

for each bigram state, we count the number of tokens that come after the bigram.  as we advance through the machine, we will move to a new state by bigram and then select a next 1-gram based on probabilities.

for example, if the bigram (a,b) is followed by (c) 3 times and followed by (d) 1 time, we would capture these counts here.

later these counts will be turned into a probability distribution to run the machine.


In [30]:
from itertools import groupby

print("now gather up the counts")
for k, g in groupby(trigrams, lambda x: (x[0], x[1])):
    # for production version, we would not make this a list here
    g = list(g)
    print(f'{k} -> {g}')
    for k2, g2 in groupby(g, lambda x: x[2]):
        # for production version, we would not make this a list here
        g2 = list(g2)
        print(f'   {k2} -> {len(g2)}')
        counts[k][k2] += len(list(g2))
    print()

counts

now gather up the counts
('a', 'b') -> [('a', 'b', 'c'), ('a', 'b', 'c'), ('a', 'b', 'e')]
   c -> 2
   e -> 1

('b', 'c') -> [('b', 'c', 'a'), ('b', 'c', 'd')]
   a -> 1
   d -> 1

('b', 'e') -> [('b', 'e', 'd')]
   d -> 1

('c', 'a') -> [('c', 'a', 'b')]
   b -> 1

('c', 'd') -> [('c', 'd', 'a')]
   a -> 1

('d', 'a') -> [('d', 'a', 'b')]
   b -> 1

('e', 'd') -> [('e', 'd', 'a')]
   a -> 1



defaultdict(<function __main__.<lambda>()>,
            {('a', 'b'): defaultdict(int, {'c': 2, 'e': 1}),
             ('b', 'c'): defaultdict(int, {'a': 1, 'd': 1}),
             ('b', 'e'): defaultdict(int, {'d': 1}),
             ('c', 'a'): defaultdict(int, {'b': 1}),
             ('c', 'd'): defaultdict(int, {'a': 1}),
             ('d', 'a'): defaultdict(int, {'b': 1}),
             ('e', 'd'): defaultdict(int, {'a': 1})})

## Step 4 - convert counts to probabilities

for each bigram state, we create a tuple of two parallel lists.

one list contains the list of continuation tokens

one list contains the list of probabilities for that continuation token

at the end of this step, we have our model, and we can use this model to generate output

In [32]:
states = {}
for k,v in counts.items():
    n = sum(v.values())
    words=[]
    probs=[]
    for k2,v2 in v.items():
        words.append(k2)
        probs.append(v2/n)
    states[k] = (words, probs)

states

{('a', 'b'): (['c', 'e'], [0.6666666666666666, 0.3333333333333333]),
 ('b', 'c'): (['a', 'd'], [0.5, 0.5]),
 ('b', 'e'): (['d'], [1.0]),
 ('c', 'a'): (['b'], [1.0]),
 ('c', 'd'): (['a'], [1.0]),
 ('d', 'a'): (['b'], [1.0]),
 ('e', 'd'): (['a'], [1.0])}

## Step 5 - generate output based on probabilities in the model

In [57]:
import random
import numpy as np

out = []
not_done = True
print("randomly choose initial state")
curr_state = random.choice(list(states.keys()))
MAX_ITERS = 20
iter = 0
while iter < MAX_ITERS:
    print(f"curr_state: {curr_state} -> {states[curr_state]}")
    print(f"append: ({curr_state[0]})")
    out.append(curr_state[0])
    randompick = np.random.multinomial(1, states[curr_state][1])
    idx = list(randompick).index(1)
    print(f"pick: ({states[curr_state][0][idx]})")
    curr_state = (curr_state[1], states[curr_state][0][idx])
    iter += 1

print(out)


randomly choose initial state
curr_state: ('d', 'a') -> (['b'], [1.0])
append: (d)
pick: (b)
curr_state: ('a', 'b') -> (['c', 'e'], [0.6666666666666666, 0.3333333333333333])
append: (a)
pick: (c)
curr_state: ('b', 'c') -> (['a', 'd'], [0.5, 0.5])
append: (b)
pick: (a)
curr_state: ('c', 'a') -> (['b'], [1.0])
append: (c)
pick: (b)
curr_state: ('a', 'b') -> (['c', 'e'], [0.6666666666666666, 0.3333333333333333])
append: (a)
pick: (c)
curr_state: ('b', 'c') -> (['a', 'd'], [0.5, 0.5])
append: (b)
pick: (d)
curr_state: ('c', 'd') -> (['a'], [1.0])
append: (c)
pick: (a)
curr_state: ('d', 'a') -> (['b'], [1.0])
append: (d)
pick: (b)
curr_state: ('a', 'b') -> (['c', 'e'], [0.6666666666666666, 0.3333333333333333])
append: (a)
pick: (c)
curr_state: ('b', 'c') -> (['a', 'd'], [0.5, 0.5])
append: (b)
pick: (a)
curr_state: ('c', 'a') -> (['b'], [1.0])
append: (c)
pick: (b)
curr_state: ('a', 'b') -> (['c', 'e'], [0.6666666666666666, 0.3333333333333333])
append: (a)
pick: (c)
curr_state: ('b', 'c') -

## Step 6 - Conclusion

This is the algorithm implemented in Ramona.

See the code in model.py for the productionized version.  
It is not very different, but does some extra work around sentence boundaries in order to generate natural sounding sentences.

Enjoy!